In [5]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import re
import pandas
import numpy as np

In [2]:
# fixes issue with autocomplete and pandas
%config Completer.use_jedi = False

In [6]:
# first get player positions
draft = pandas.DataFrame()

indx = 0
n_rounds = 16
# iterate through rounds
for rnd in range(1,n_rounds+1):
    print('working on round',rnd)
    url = "https://canada.fantasy.nfl.com/league/6787729/draftresults?draftResultsDetail=%s&draftResultsTab=round&draftResultsType=results"%rnd
    #print(url)
    page = urlopen(url)
    html = page.read()
    page.close()
    canned = bs(html, 'html.parser')
    # super janky way to get the picks using the order of presentation
    pick = 1
    i = 0
    for tag in canned.find_all(re.compile("^a")):
        jnk = tag.decode_contents()
        if '<' in jnk or 'View'  in jnk: continue
        if i % 2 == 0:
            draft.loc[indx,'round'] = rnd
            draft.loc[indx,'pick'] = pick
            draft.loc[indx,'player'] = jnk
            i += 1
        else:
            draft.loc[indx,'team'] = jnk
            i += 1
            pick += 1
            indx += 1

working on round 1
working on round 2
working on round 3
working on round 4
working on round 5
working on round 6
working on round 7
working on round 8
working on round 9
working on round 10
working on round 11
working on round 12
working on round 13
working on round 14
working on round 15
working on round 16


In [7]:
# now rinse and repeat to get the team and position
indx = 0
n_rounds = 16
for rnd in range(1,n_rounds+1):
    print('working on round',rnd)
    url = "https://canada.fantasy.nfl.com/league/6787729/draftresults?draftResultsDetail=%s&draftResultsTab=round&draftResultsType=results"%rnd
    #print(url)
    page = urlopen(url)
    html = page.read()
    page.close()
    canned = bs(html, 'html.parser')
    pick = 1
    i = 0
    for tag in canned.find_all(re.compile("^em")):
        jnk = tag.decode_contents()
        draft.loc[indx,'info'] = jnk
        indx += 1

working on round 1
working on round 2
working on round 3
working on round 4
working on round 5
working on round 6
working on round 7
working on round 8
working on round 9
working on round 10
working on round 11
working on round 12
working on round 13
working on round 14
working on round 15
working on round 16


In [8]:
draft.loc[:,'overall'] = np.array(draft.index)+1
draft

,round,pick,player,team,info,overall
0,1.0,1.0,Christian McCaffrey,The Longest Yard,RB - CAR,1
1,1.0,2.0,Saquon Barkley,Tight Ends Spread Sheets,RB - NYG,2
2,1.0,3.0,Ezekiel Elliott,Eatin W's,RB - DAL,3
3,1.0,4.0,Alvin Kamara,ganggreen215,RB - NO,4
4,1.0,5.0,Dalvin Cook,Gronk's New Groove,RB - MIN,5
...,...,...,...,...,...,...
155,16.0,6.0,Brandon Aiyuk,Gronk's New Groove,WR - SF,156
156,16.0,7.0,Michael Pittman,ganggreen215,WR - IND,157
157,16.0,8.0,Parris Campbell,Eatin W's,WR - IND,158
158,16.0,9.0,Noah Fant,Tight Ends Spread Sheets,TE - DEN,159


In [9]:
def get_unique_str(row):
    info = row['info']
    if 'DEF' in info:
        nsplit = row['player'].split(' ')
        if 'Football Team' in row['player']:
            pstr = '%s %s DEF'%(nsplit[-2],nsplit[-1])
        else:
            pstr = '%s DEF '%(nsplit[-1])
    else:
        if ' - ' in info:
            position, team = info.split(' - ')
        else:
            position = info
            team = 'Free Agent'
        nsplit = row['player'].split(' ')
        name = ''
        for i in range(1,len(nsplit)):
            if i == len(nsplit)-1:
                name += '%s, '%nsplit[i]
            else:
                name += '%s '%nsplit[i]
        name += '%s.'%nsplit[0][0]
        pstr = '%s %s %s'%(name,position,team)
    
    return pstr

In [10]:
for i,row in draft.iterrows():
    pstr = get_unique_str(row)
    draft.loc[:,'pstr'] = draft.loc[i,'unique_str'] = pstr

In [12]:
draft.head()

,round,pick,player,team,info,overall,pstr,unique_str
0,1.0,1.0,Christian McCaffrey,The Longest Yard,RB - CAR,1,"Goedert, D. TE PHI","McCaffrey, C. RB CAR"
1,1.0,2.0,Saquon Barkley,Tight Ends Spread Sheets,RB - NYG,2,"Goedert, D. TE PHI","Barkley, S. RB NYG"
2,1.0,3.0,Ezekiel Elliott,Eatin W's,RB - DAL,3,"Goedert, D. TE PHI","Elliott, E. RB DAL"
3,1.0,4.0,Alvin Kamara,ganggreen215,RB - NO,4,"Goedert, D. TE PHI","Kamara, A. RB NO"
4,1.0,5.0,Dalvin Cook,Gronk's New Groove,RB - MIN,5,"Goedert, D. TE PHI","Cook, D. RB MIN"


In [13]:
draft.to_csv('DraftOrder.csv')